# CPLEX IMPLEMENTATION

## Imports

In [2]:
import cplex
from cplex import Cplex
from cplex import SparsePair

import os
import re
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt

## Define fixed sets: Time periods and road Widths

In [3]:
# Define fixed sets
T = tuple([1, 2, 3, 4, 5])  # 5 time periods
W = tuple(['timber','wide'])     # Road widths

## component-dependent sets

In [4]:
component_dir =r'1_Preprocessed_Data\3_Road_Network_Graphs\component_10'

### Load nodes

In [5]:
file_path = f'{component_dir}\\4withsources_nodes.csv'
df = pd.read_csv(file_path)
df

,x,y,is_exit,is_source,stands
0,-15106.71098,150581.22331,False,False,"[945, 948]"
1,-16216.49208,150851.53241,True,False,[946]
2,-15464.13998,151201.64051,False,False,"[944, 1304, 947]"
3,-15761.71028,151228.89011,True,False,[944]
4,-15587.95728,151189.65731,False,False,"[944, 1304]"
5,-14863.04848,150875.44951,False,False,"[945, 942]"
6,-15443.44238,150603.44861,False,False,"[945, 947, 948]"
7,-15774.06238,150740.86021,False,False,"[944, 946, 947]"
8,-14861.44738,150934.48421,False,False,"[1304, 942]"
9,-15848.72048,150337.44601,False,False,"[947, 948]"


### V = all nodes

In [6]:
V = list(zip(df['x'], df['y'])) #csv has two columns x and y
V

[(-15106.71098, 150581.22331),
 (-16216.49208, 150851.53241),
 (-15464.13998, 151201.64051),
 (-15761.71028, 151228.89011),
 (-15587.95728, 151189.65731),
 (-14863.04848, 150875.44951),
 (-15443.44238, 150603.44861),
 (-15774.06238, 150740.86021),
 (-14861.44738, 150934.48421),
 (-15848.72048, 150337.44601),
 (-15387.94528, 151109.43011),
 (-15343.90788, 150903.59781),
 (-15969.22818, 150361.86671),
 (-15945.09588, 150839.36401),
 (-15069.81928, 150869.45261),
 (-15230.13938, 150665.38771),
 (-15369.24588, 150888.48011),
 (-15334.38388, 150628.08511),
 (-15842.48604, 151170.45481),
 (-16192.43465, 150699.23742),
 (-15289.32593, 150766.1793),
 (-15189.10959, 150864.98136),
 (-15172.98507, 151000.47245),
 (-15736.21395, 150994.90272),
 (-15057.87723, 150708.68595),
 (-16015.52817, 150676.42039),
 (-15581.13059, 150734.95184),
 (-15471.76829, 150383.40184),
 (-15370.22923, 151382.12787)]

### Map nodes to short names

In [7]:
# Create a dictionary mapping each node to a short name
node_mapping = {f'node{i + 1}': V[i] for i in range(len(V))}

# Print the mapping
print(node_mapping)

{'node1': (-15106.71098, 150581.22331), 'node2': (-16216.49208, 150851.53241), 'node3': (-15464.13998, 151201.64051), 'node4': (-15761.71028, 151228.89011), 'node5': (-15587.95728, 151189.65731), 'node6': (-14863.04848, 150875.44951), 'node7': (-15443.44238, 150603.44861), 'node8': (-15774.06238, 150740.86021), 'node9': (-14861.44738, 150934.48421), 'node10': (-15848.72048, 150337.44601), 'node11': (-15387.94528, 151109.43011), 'node12': (-15343.90788, 150903.59781), 'node13': (-15969.22818, 150361.86671), 'node14': (-15945.09588, 150839.36401), 'node15': (-15069.81928, 150869.45261), 'node16': (-15230.13938, 150665.38771), 'node17': (-15369.24588, 150888.48011), 'node18': (-15334.38388, 150628.08511), 'node19': (-15842.48604, 151170.45481), 'node20': (-16192.43465, 150699.23742), 'node21': (-15289.32593, 150766.1793), 'node22': (-15189.10959, 150864.98136), 'node23': (-15172.98507, 151000.47245), 'node24': (-15736.21395, 150994.90272), 'node25': (-15057.87723, 150708.68595), 'node26':

In [8]:
reversed_node_mapping = {v: k for k, v in node_mapping.items()}
reversed_node_mapping

{(-15106.71098, 150581.22331): 'node1',
 (-16216.49208, 150851.53241): 'node2',
 (-15464.13998, 151201.64051): 'node3',
 (-15761.71028, 151228.89011): 'node4',
 (-15587.95728, 151189.65731): 'node5',
 (-14863.04848, 150875.44951): 'node6',
 (-15443.44238, 150603.44861): 'node7',
 (-15774.06238, 150740.86021): 'node8',
 (-14861.44738, 150934.48421): 'node9',
 (-15848.72048, 150337.44601): 'node10',
 (-15387.94528, 151109.43011): 'node11',
 (-15343.90788, 150903.59781): 'node12',
 (-15969.22818, 150361.86671): 'node13',
 (-15945.09588, 150839.36401): 'node14',
 (-15069.81928, 150869.45261): 'node15',
 (-15230.13938, 150665.38771): 'node16',
 (-15369.24588, 150888.48011): 'node17',
 (-15334.38388, 150628.08511): 'node18',
 (-15842.48604, 151170.45481): 'node19',
 (-16192.43465, 150699.23742): 'node20',
 (-15289.32593, 150766.1793): 'node21',
 (-15189.10959, 150864.98136): 'node22',
 (-15172.98507, 151000.47245): 'node23',
 (-15736.21395, 150994.90272): 'node24',
 (-15057.87723, 150708.685

In [9]:
V = [reversed_node_mapping[v] for v in V]
V

['node1',
 'node2',
 'node3',
 'node4',
 'node5',
 'node6',
 'node7',
 'node8',
 'node9',
 'node10',
 'node11',
 'node12',
 'node13',
 'node14',
 'node15',
 'node16',
 'node17',
 'node18',
 'node19',
 'node20',
 'node21',
 'node22',
 'node23',
 'node24',
 'node25',
 'node26',
 'node27',
 'node28',
 'node29']

### Arcs & Edges

In [10]:
# Load the arcs from the CSV files
arcs_fw_df = pd.read_csv(f'{component_dir}/arcsforward.csv')
arcs_bw_df = pd.read_csv(f'{component_dir}/arcsbackward.csv')
arcs_df = pd.read_csv(f'{component_dir}/arcs.csv')

# Assign the data to the respective variables
A = arcs_df.values
print(len(A), len(arcs_fw_df),len(arcs_bw_df))
A

146 73 73


array([['(-15106.71098, 150581.22331)', '(-15443.44238, 150603.44861)'],
       ['(-15106.71098, 150581.22331)', '(-15848.72048, 150337.44601)'],
       ['(-15106.71098, 150581.22331)', '(-14863.04848, 150875.44951)'],
       ['(-15106.71098, 150581.22331)', '(-15057.87723, 150708.68595)'],
       ['(-15106.71098, 150581.22331)', '(-15471.76829, 150383.40184)'],
       ['(-15443.44238, 150603.44861)', '(-15106.71098, 150581.22331)'],
       ['(-15443.44238, 150603.44861)', '(-15848.72048, 150337.44601)'],
       ['(-15443.44238, 150603.44861)', '(-15334.38388, 150628.08511)'],
       ['(-15443.44238, 150603.44861)', '(-15057.87723, 150708.68595)'],
       ['(-15443.44238, 150603.44861)', '(-15581.13059, 150734.95184)'],
       ['(-15443.44238, 150603.44861)', '(-15471.76829, 150383.40184)'],
       ['(-15848.72048, 150337.44601)', '(-15106.71098, 150581.22331)'],
       ['(-15848.72048, 150337.44601)', '(-15443.44238, 150603.44861)'],
       ['(-15848.72048, 150337.44601)', '(-15969.22

#### verify set of arcs separation

In [11]:
# Convert arcs to sets of tuples for easy comparison
A_fw = {tuple(arc) for arc in arcs_fw_df.values}
A_bw = {tuple(arc) for arc in arcs_bw_df.values}

# Check for missing backward arcs
missing_bw = []
for arc in A_fw:
    reverse_arc = (arc[1], arc[0])
    if reverse_arc not in A_bw:
        missing_bw.append((arc, reverse_arc))

if missing_bw:
    print("⚠️ Some forward arcs have no reverse in A_bw:")
    for fw, expected_bw in missing_bw:
        print(f"Missing backward arc for forward arc {fw}: expected {expected_bw}")
else:
    print("✅ All forward arcs have corresponding backward arcs.")


✅ All forward arcs have corresponding backward arcs.


In [12]:
edges = A_fw
edges

{('(-14861.44738, 150934.48421)', '(-15172.98507, 151000.47245)'),
 ('(-14861.44738, 150934.48421)', '(-15370.22923, 151382.12787)'),
 ('(-14861.44738, 150934.48421)', '(-15387.94528, 151109.43011)'),
 ('(-14863.04848, 150875.44951)', '(-14861.44738, 150934.48421)'),
 ('(-14863.04848, 150875.44951)', '(-15057.87723, 150708.68595)'),
 ('(-14863.04848, 150875.44951)', '(-15069.81928, 150869.45261)'),
 ('(-14863.04848, 150875.44951)', '(-15172.98507, 151000.47245)'),
 ('(-15069.81928, 150869.45261)', '(-15057.87723, 150708.68595)'),
 ('(-15069.81928, 150869.45261)', '(-15172.98507, 151000.47245)'),
 ('(-15069.81928, 150869.45261)', '(-15189.10959, 150864.98136)'),
 ('(-15069.81928, 150869.45261)', '(-15230.13938, 150665.38771)'),
 ('(-15106.71098, 150581.22331)', '(-14863.04848, 150875.44951)'),
 ('(-15106.71098, 150581.22331)', '(-15057.87723, 150708.68595)'),
 ('(-15106.71098, 150581.22331)', '(-15443.44238, 150603.44861)'),
 ('(-15106.71098, 150581.22331)', '(-15471.76829, 150383.40184

#### map arcs to short names

In [13]:
# Function to map coordinates to short names
def map_coordinates_to_short_name(coords):
    for short_name, node_coords in node_mapping.items():
        if tuple(map(float, coords.strip('()').split(', '))) == node_coords:
            return short_name
    return None  # Return None if the coordinates are not found

# Replace coordinates with their short names
A = [[map_coordinates_to_short_name(pair[0]), map_coordinates_to_short_name(pair[1])] for pair in A]
A_fw = [[map_coordinates_to_short_name(pair[0]), map_coordinates_to_short_name(pair[1])] for pair in A_fw]
A_bw = [[map_coordinates_to_short_name(pair[0]), map_coordinates_to_short_name(pair[1])] for pair in A_bw]
A_fw[1]

['node11', 'node17']

In [14]:
A = tuple("".join(pair) for pair in A)
A_fw = tuple("".join(pair) for pair in A_fw)
A_bw = tuple("".join(pair) for pair in A_bw)
E = A_fw
A_fw[1]

'node11node17'

#### verify fw bw arcs seperation

In [15]:
print(set(A) == set(A_fw) | set(A_bw))  # Should be True if they match

True


### helper function swap halves

In [16]:
# Helper function to swap halves
def swap_direction(arc):
    mid = arc.find('node', 4)
    return arc[mid:] + arc[:mid]

# Create the mapping from forward to backward (using half-swap)
fw_to_bw_map = {}
for arc in A_fw:
    swapped = swap_direction(arc)
    if swapped in A_bw:
        fw_to_bw_map[arc] = swapped

# Optional: backward to forward
bw_to_fw_map = {v: k for k, v in fw_to_bw_map.items()}
bw_to_fw_map

{'node20node2': 'node2node20',
 'node17node11': 'node11node17',
 'node21node12': 'node12node21',
 'node22node12': 'node12node22',
 'node8node3': 'node3node8',
 'node27node17': 'node17node27',
 'node24node3': 'node3node24',
 'node14node2': 'node2node14',
 'node25node7': 'node7node25',
 'node29node9': 'node9node29',
 'node26node2': 'node2node26',
 'node9node6': 'node6node9',
 'node27node7': 'node7node27',
 'node15node12': 'node12node15',
 'node11node9': 'node9node11',
 'node25node6': 'node6node25',
 'node27node13': 'node13node27',
 'node23node15': 'node15node23',
 'node28node7': 'node7node28',
 'node29node3': 'node3node29',
 'node27node3': 'node3node27',
 'node16node15': 'node15node16',
 'node18node17': 'node17node18',
 'node18node7': 'node7node18',
 'node11node3': 'node3node11',
 'node20node13': 'node13node20',
 'node24node14': 'node14node24',
 'node26node20': 'node20node26',
 'node23node12': 'node12node23',
 'node5node3': 'node3node5',
 'node27node10': 'node10node27',
 'node16node12': 

### V_source and its arcs

In [17]:
# load source nodes
file_path = f'{component_dir}\\source_nodes.csv'
df = pd.read_csv(file_path)
df

,x,y,ID_UG
0,-15289.32593,150766.17930,1469
1,-15189.10959,150864.98136,1470
2,-15172.98507,151000.47245,942
3,-15736.21395,150994.90272,944
4,-15057.87723,150708.68595,945
5,-16015.52817,150676.42039,946
6,-15581.13059,150734.95184,947
7,-15471.76829,150383.40184,948
8,-15370.22923,151382.12787,1304


In [18]:
V_source = { int(row['ID_UG']) : (row['x'], row['y']) for _, row in df.iterrows() }
V_source

{1469: (-15289.32593, 150766.1793),
 1470: (-15189.10959, 150864.98136),
 942: (-15172.98507, 151000.47245),
 944: (-15736.21395, 150994.90272),
 945: (-15057.87723, 150708.68595),
 946: (-16015.52817, 150676.42039),
 947: (-15581.13059, 150734.95184),
 948: (-15471.76829, 150383.40184),
 1304: (-15370.22923, 151382.12787)}

In [19]:
for key, node in V_source.items():
    V_source[key] = reversed_node_mapping[node]
V_source

{1469: 'node21',
 1470: 'node22',
 942: 'node23',
 944: 'node24',
 945: 'node25',
 946: 'node26',
 947: 'node27',
 948: 'node28',
 1304: 'node29'}

### S = stands in V_source

In [20]:
S = set(V_source.keys())
S

{942, 944, 945, 946, 947, 948, 1304, 1469, 1470}

### Ingoing arcs to source nodes

In [21]:
# create the set of arcs ingoing to source nodes
source_arcs_ingoing = {}

for key, s in V_source.items():
    ingoing = set()
    for arc in A:
        if arc.endswith(s):
            ingoing.add(arc)
    source_arcs_ingoing[key] = ingoing

total_arcs = sum(len(v) for v in source_arcs_ingoing.values())
print(total_arcs, 'ingoing to source nodes in total')
source_arcs_ingoing    

45 ingoing to source nodes in total


{1469: {'node12node21', 'node16node21', 'node17node21', 'node18node21'},
 1470: {'node12node22', 'node15node22', 'node16node22'},
 942: {'node11node23',
  'node12node23',
  'node15node23',
  'node17node23',
  'node6node23',
  'node9node23'},
 944: {'node14node24',
  'node19node24',
  'node3node24',
  'node4node24',
  'node5node24',
  'node8node24'},
 945: {'node15node25',
  'node16node25',
  'node18node25',
  'node1node25',
  'node6node25',
  'node7node25'},
 946: {'node13node26',
  'node14node26',
  'node20node26',
  'node2node26',
  'node8node26'},
 947: {'node10node27',
  'node11node27',
  'node13node27',
  'node17node27',
  'node18node27',
  'node3node27',
  'node7node27',
  'node8node27'},
 948: {'node10node28', 'node1node28', 'node7node28'},
 1304: {'node11node29', 'node3node29', 'node5node29', 'node9node29'}}

#### Outgoing arcs from source nodes

In [22]:
# create the set of arcs outgoing from source nodes
source_arcs_outgoing = {}

for key, s in V_source.items():
    outgoing = set()
    for arc in A:
        second_node_index = arc.find("node", 4)  # skip the first "node"
        first_node = arc[:second_node_index]
        if first_node == s:
            outgoing.add(arc)
    source_arcs_outgoing[key] = outgoing

total_arcs = sum(len(v) for v in source_arcs_outgoing.values())
print(total_arcs)    
source_arcs_outgoing          

45


{1469: {'node21node12', 'node21node16', 'node21node17', 'node21node18'},
 1470: {'node22node12', 'node22node15', 'node22node16'},
 942: {'node23node11',
  'node23node12',
  'node23node15',
  'node23node17',
  'node23node6',
  'node23node9'},
 944: {'node24node14',
  'node24node19',
  'node24node3',
  'node24node4',
  'node24node5',
  'node24node8'},
 945: {'node25node1',
  'node25node15',
  'node25node16',
  'node25node18',
  'node25node6',
  'node25node7'},
 946: {'node26node13',
  'node26node14',
  'node26node2',
  'node26node20',
  'node26node8'},
 947: {'node27node10',
  'node27node11',
  'node27node13',
  'node27node17',
  'node27node18',
  'node27node3',
  'node27node7',
  'node27node8'},
 948: {'node28node1', 'node28node10', 'node28node7'},
 1304: {'node29node11', 'node29node3', 'node29node5', 'node29node9'}}

### Exit Nodes and exit arcs

In [23]:
# load exit nodes
file_path = f'{component_dir}\\exit_nodes.csv'
df = pd.read_csv(file_path)
V_exit = list(zip(df['x'], df['y'])) #csv has two columns x and y
V_exit

[(-16216.49208, 150851.53241),
 (-15761.71028, 151228.89011),
 (-15842.48604, 151170.45481),
 (-16192.43465, 150699.23742)]

In [24]:
V_exit = [short_name for node in V_exit for short_name, coords in node_mapping.items() if coords == node]
V_exit

['node2', 'node4', 'node19', 'node20']

#### Exit arcs outgoing

In [25]:
# create the set of exit arcs outgoing from exit nodes
exit_arcs_outgoing = {}

for s in V_exit:
    outgoing = set()
    for arc in A:
        second_node_index = arc.find("node", 4)  # skip the first "node"
        first_node = arc[:second_node_index]
        if first_node == s:
            outgoing.add(arc)
    exit_arcs_outgoing[s] = outgoing  # Assign the set to the dict

total_arcs = sum(len(v) for v in exit_arcs_outgoing.values())
print(total_arcs)
exit_arcs_outgoing    

12


{'node2': {'node2node14', 'node2node20', 'node2node26'},
 'node4': {'node4node19', 'node4node24', 'node4node5'},
 'node19': {'node19node14', 'node19node24', 'node19node4'},
 'node20': {'node20node13', 'node20node2', 'node20node26'}}

#### Exit arcs ingoing

In [26]:
# create the set of exit arcs ingoing to exit nodes
exit_arcs_ingoing = {}

for s in V_exit:
    ingoing = set()
    for arc in A:
        if arc.endswith(s):
            ingoing.add(arc)
    exit_arcs_ingoing[s] = ingoing        

total_arcs = sum(len(v) for v in exit_arcs_ingoing.values())
print(total_arcs)
exit_arcs_ingoing

12


{'node2': {'node14node2', 'node20node2', 'node26node2'},
 'node4': {'node19node4', 'node24node4', 'node5node4'},
 'node19': {'node14node19', 'node24node19', 'node4node19'},
 'node20': {'node13node20', 'node26node20', 'node2node20'}}

### Transit nodes 

In [27]:
V_transit = set(V) - set(V_source.values()) - set(V_exit)
print(len(V)-len(set(V_source.values())) - len(set(V_exit)))
print(len(V_transit))
V_transit

16
16


{'node1',
 'node10',
 'node11',
 'node12',
 'node13',
 'node14',
 'node15',
 'node16',
 'node17',
 'node18',
 'node3',
 'node5',
 'node6',
 'node7',
 'node8',
 'node9'}

#### Ingoing/ Outgoing per transit node

In [28]:
ingoing_per_transitnode = {}
outgoing_per_transitnode = {}

for tnode in V_transit:
    ingoing = set()
    outgoing = set()
    for arc in A:
        if arc.endswith(tnode):
            ingoing.add(arc)
            outgoing.add(swap_direction(arc))
    ingoing_per_transitnode[tnode] = ingoing
    outgoing_per_transitnode[tnode] = outgoing

total_ingoing = sum(len(v) for v in ingoing_per_transitnode.values())
print(total_ingoing)

total_outgoing = sum(len(v) for v in outgoing_per_transitnode.values())
print(total_outgoing)
ingoing_per_transitnode

89
89


{'node3': {'node11node3',
  'node24node3',
  'node27node3',
  'node29node3',
  'node5node3',
  'node8node3'},
 'node16': {'node12node16',
  'node15node16',
  'node18node16',
  'node21node16',
  'node22node16',
  'node25node16'},
 'node1': {'node10node1',
  'node25node1',
  'node28node1',
  'node6node1',
  'node7node1'},
 'node5': {'node24node5',
  'node29node5',
  'node3node5',
  'node4node5',
  'node9node5'},
 'node11': {'node17node11',
  'node23node11',
  'node27node11',
  'node29node11',
  'node3node11',
  'node9node11'},
 'node18': {'node16node18',
  'node17node18',
  'node21node18',
  'node25node18',
  'node27node18',
  'node7node18'},
 'node7': {'node10node7',
  'node18node7',
  'node1node7',
  'node25node7',
  'node27node7',
  'node28node7'},
 'node9': {'node11node9',
  'node23node9',
  'node29node9',
  'node5node9',
  'node6node9'},
 'node8': {'node13node8',
  'node14node8',
  'node24node8',
  'node26node8',
  'node27node8',
  'node3node8'},
 'node10': {'node13node10',
  'node1

### Bonus: Load Boundary nodes per stand

In [29]:
# load boundaries
boundaries_dict = {}

with open(f'{component_dir}\\boundaries.csv', 'r') as file:
    # Skip the header line
    next(file)
    for line in file:
        id_ug, nodes = line.strip().split(';')
        # Keep nodes as strings with parentheses intact
        boundaries_dict[id_ug] = [node.strip() for node in nodes.split(',')]

boundaries_dict

{'945': ['(-15106.71098',
  '150581.22331)',
  '(-14863.04848',
  '150875.44951)',
  '(-15443.44238',
  '150603.44861)',
  '(-15069.81928',
  '150869.45261)',
  '(-15230.13938',
  '150665.38771)',
  '(-15334.38388',
  '150628.08511)',
  '(-15057.87723',
  '150708.68595)'],
 '948': ['(-15106.71098',
  '150581.22331)',
  '(-15443.44238',
  '150603.44861)',
  '(-15848.72048',
  '150337.44601)',
  '(-15471.76829',
  '150383.40184)'],
 '946': ['(-16216.49208',
  '150851.53241)',
  '(-15774.06238',
  '150740.86021)',
  '(-15969.22818',
  '150361.86671)',
  '(-15945.09588',
  '150839.36401)',
  '(-16192.43465',
  '150699.23742)',
  '(-16015.52817',
  '150676.42039)'],
 '944': ['(-15464.13998',
  '151201.64051)',
  '(-15761.71028',
  '151228.89011)',
  '(-15587.95728',
  '151189.65731)',
  '(-15774.06238',
  '150740.86021)',
  '(-15945.09588',
  '150839.36401)',
  '(-15842.48604',
  '151170.45481)',
  '(-15736.21395',
  '150994.90272)'],
 '1304': ['(-15464.13998',
  '151201.64051)',
  '(-15587

In [30]:
# Reverse mapping: string of coords → short name
reverse_mapping = {
    f'({x[0]}, {x[1]})': short_name
    for short_name, x in node_mapping.items()
}

In [31]:
# Step 2: Recombine and map
mapped_boundaries_dict = {}

for ug_id, node_parts in boundaries_dict.items():
    # Rebuild coordinate strings from every 2 parts
    rebuilt_coords = [f'{node_parts[i].strip()}, {node_parts[i+1].strip()}' for i in range(0, len(node_parts), 2)]
    
    # Add parentheses back if needed
    rebuilt_coords = [f'({coord})' if not coord.startswith('(') else coord for coord in rebuilt_coords]
    
    # Map to short names using reverse_mapping
    mapped_nodes = [reverse_mapping.get(coord, None) for coord in rebuilt_coords]
    
    mapped_boundaries_dict[ug_id] = mapped_nodes
mapped_boundaries_dict    

{'945': ['node1', 'node6', 'node7', 'node15', 'node16', 'node18', 'node25'],
 '948': ['node1', 'node7', 'node10', 'node28'],
 '946': ['node2', 'node8', 'node13', 'node14', 'node20', 'node26'],
 '944': ['node3', 'node4', 'node5', 'node8', 'node14', 'node19', 'node24'],
 '1304': ['node3', 'node5', 'node9', 'node11', 'node29'],
 '947': ['node3',
  'node7',
  'node8',
  'node10',
  'node11',
  'node13',
  'node17',
  'node18',
  'node27'],
 '942': ['node6', 'node9', 'node11', 'node12', 'node15', 'node17', 'node23'],
 '1469': ['node12', 'node16', 'node17', 'node18', 'node21'],
 '1470': ['node12', 'node15', 'node16', 'node22']}

## Load parameters and actual values

### Load Costs

In [130]:
# load costs
edge_attr_df = pd.read_csv(f'{component_dir}/4withsources_edges_with_attributes.csv')
edge_attr_df

,"Node1(x,y)","Node2(x,y)",edgelength,has_exit,has_source,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade,slope,removed_nodes
0,"(-15106.71098, 150581.22331)","(-15443.44238, 150603.44861)",0.382454,False,False,1847,719,2772,719,1386,False,"[(-15167.38228, 150601.09241), (-15256.60638, ..."
1,"(-15106.71098, 150581.22331)","(-15848.72048, 150337.44601)",1.344332,False,False,6491,2520,9740,2520,4868,False,"[(-15470.29388, 150195.65221), (-15461.41768, ..."
2,"(-15106.71098, 150581.22331)","(-14863.04848, 150875.44951)",0.511532,False,False,2471,960,3707,960,1854,False,"[(-15060.68178, 150574.20781), (-14995.52008, ..."
3,"(-15106.71098, 150581.22331)","(-15057.87723, 150708.68595)",0.000000,False,True,0,0,0,0,0,False,NaN
4,"(-15106.71098, 150581.22331)","(-15471.76829, 150383.40184)",0.000000,False,True,0,0,0,0,0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
68,"(-15334.38388, 150628.08511)","(-15289.32593, 150766.1793)",0.000000,False,True,0,0,0,0,0,False,NaN
69,"(-15334.38388, 150628.08511)","(-15057.87723, 150708.68595)",0.000000,False,True,0,0,0,0,0,False,NaN
70,"(-15334.38388, 150628.08511)","(-15581.13059, 150734.95184)",0.000000,False,True,0,0,0,0,0,False,NaN
71,"(-15842.48604, 151170.45481)","(-15736.21395, 150994.90272)",0.000000,True,True,0,0,0,0,0,False,NaN


In [131]:
# Map the Node1 and Node2 columns using the reverse mapping
edge_attr_df['node1_ID'] = edge_attr_df['Node1(x,y)'].map(reverse_mapping)
edge_attr_df['node2_ID'] = edge_attr_df['Node2(x,y)'].map(reverse_mapping)
edge_attr_df

,"Node1(x,y)","Node2(x,y)",edgelength,has_exit,has_source,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade,slope,removed_nodes,node1_ID,node2_ID
0,"(-15106.71098, 150581.22331)","(-15443.44238, 150603.44861)",0.382454,False,False,1847,719,2772,719,1386,False,"[(-15167.38228, 150601.09241), (-15256.60638, ...",node1,node7
1,"(-15106.71098, 150581.22331)","(-15848.72048, 150337.44601)",1.344332,False,False,6491,2520,9740,2520,4868,False,"[(-15470.29388, 150195.65221), (-15461.41768, ...",node1,node10
2,"(-15106.71098, 150581.22331)","(-14863.04848, 150875.44951)",0.511532,False,False,2471,960,3707,960,1854,False,"[(-15060.68178, 150574.20781), (-14995.52008, ...",node1,node6
3,"(-15106.71098, 150581.22331)","(-15057.87723, 150708.68595)",0.000000,False,True,0,0,0,0,0,False,NaN,node1,node25
4,"(-15106.71098, 150581.22331)","(-15471.76829, 150383.40184)",0.000000,False,True,0,0,0,0,0,False,NaN,node1,node28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"(-15334.38388, 150628.08511)","(-15289.32593, 150766.1793)",0.000000,False,True,0,0,0,0,0,False,NaN,node18,node21
69,"(-15334.38388, 150628.08511)","(-15057.87723, 150708.68595)",0.000000,False,True,0,0,0,0,0,False,NaN,node18,node25
70,"(-15334.38388, 150628.08511)","(-15581.13059, 150734.95184)",0.000000,False,True,0,0,0,0,0,False,NaN,node18,node27
71,"(-15842.48604, 151170.45481)","(-15736.21395, 150994.90272)",0.000000,True,True,0,0,0,0,0,False,NaN,node19,node24


In [132]:
# Make sure both columns are strings and create EdgeID
edge_attr_df['EdgeID'] = edge_attr_df['node1_ID'].astype(str) + edge_attr_df['node2_ID'].astype(str)
edge_attr_df

,"Node1(x,y)","Node2(x,y)",edgelength,has_exit,has_source,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade,slope,removed_nodes,node1_ID,node2_ID,EdgeID
0,"(-15106.71098, 150581.22331)","(-15443.44238, 150603.44861)",0.382454,False,False,1847,719,2772,719,1386,False,"[(-15167.38228, 150601.09241), (-15256.60638, ...",node1,node7,node1node7
1,"(-15106.71098, 150581.22331)","(-15848.72048, 150337.44601)",1.344332,False,False,6491,2520,9740,2520,4868,False,"[(-15470.29388, 150195.65221), (-15461.41768, ...",node1,node10,node1node10
2,"(-15106.71098, 150581.22331)","(-14863.04848, 150875.44951)",0.511532,False,False,2471,960,3707,960,1854,False,"[(-15060.68178, 150574.20781), (-14995.52008, ...",node1,node6,node1node6
3,"(-15106.71098, 150581.22331)","(-15057.87723, 150708.68595)",0.000000,False,True,0,0,0,0,0,False,NaN,node1,node25,node1node25
4,"(-15106.71098, 150581.22331)","(-15471.76829, 150383.40184)",0.000000,False,True,0,0,0,0,0,False,NaN,node1,node28,node1node28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"(-15334.38388, 150628.08511)","(-15289.32593, 150766.1793)",0.000000,False,True,0,0,0,0,0,False,NaN,node18,node21,node18node21
69,"(-15334.38388, 150628.08511)","(-15057.87723, 150708.68595)",0.000000,False,True,0,0,0,0,0,False,NaN,node18,node25,node18node25
70,"(-15334.38388, 150628.08511)","(-15581.13059, 150734.95184)",0.000000,False,True,0,0,0,0,0,False,NaN,node18,node27,node18node27
71,"(-15842.48604, 151170.45481)","(-15736.21395, 150994.90272)",0.000000,True,True,0,0,0,0,0,False,NaN,node19,node24,node19node24


### Set costs parameters

In [133]:
# Initialize cost dictionaries without t
CostC = {(e, w): None for e in E for w in W}
CostM = {(e, w): None for e in E for w in W}
CostU = {e: None for e in E}

# Iterate over the DataFrame rows to assign actual cost values (rounded to 2 decimals)
for _, row in edge_attr_df.iterrows():
    e = (row['EdgeID'])  # Create edge tuple
    
    CostC[(e, W[0])] = round(row['Build5m'])
    CostC[(e, W[1])] = round(row['Build10m'])
    CostM[(e, W[0])] = round(row['Maintain5m'])
    CostM[(e, W[1])] = round(row['Maintain10m'])
    CostU[e] = round(row['Upgrade'])
CostC 

{('node2node20', 'timber'): 3120,
 ('node2node20', 'wide'): 4677,
 ('node11node17', 'timber'): 2998,
 ('node11node17', 'wide'): 4495,
 ('node12node21', 'timber'): 0,
 ('node12node21', 'wide'): 0,
 ('node12node22', 'timber'): 0,
 ('node12node22', 'wide'): 0,
 ('node3node8', 'timber'): 2980,
 ('node3node8', 'wide'): 4472,
 ('node17node27', 'timber'): 0,
 ('node17node27', 'wide'): 0,
 ('node3node24', 'timber'): 0,
 ('node3node24', 'wide'): 0,
 ('node2node14', 'timber'): 3041,
 ('node2node14', 'wide'): 4562,
 ('node7node25', 'timber'): 0,
 ('node7node25', 'wide'): 0,
 ('node9node29', 'timber'): 0,
 ('node9node29', 'wide'): 0,
 ('node2node26', 'timber'): 0,
 ('node2node26', 'wide'): 0,
 ('node6node9', 'timber'): 285,
 ('node6node9', 'wide'): 428,
 ('node7node27', 'timber'): 0,
 ('node7node27', 'wide'): 0,
 ('node12node15', 'timber'): 1845,
 ('node12node15', 'wide'): 2770,
 ('node9node11', 'timber'): 2919,
 ('node9node11', 'wide'): 4378,
 ('node6node25', 'timber'): 0,
 ('node6node25', 'wide'

### Load Access Needs

In [36]:
# load fire resistance and timber matrix
mrespath = r'1_Preprocessed_Data\0_Access_Requirements_Matrices\mres_stands_access_matrix.csv'
mwoodpath = r'1_Preprocessed_Data\0_Access_Requirements_Matrices\mwood_stands_access_matrix.csv'
stakepath = r'1_Preprocessed_Data\0_Access_Requirements_Matrices\stake_stands_access_matrix.csv'"gd prep outtakes.ipynb"

accessneeds_df = pd.read_csv(f'{mrespath}')
accessneeds_df.set_index('ID_UG', inplace=True)
accessneeds_df

,5mt1,5mt2,5mt3,5mt4,5mt5,10mt1,10mt2,10mt3,10mt4,10mt5
ID_UG,,,,,,,,,,
840,1,0,1,1,1,0,0,0,0,0
841,1,1,0,1,1,0,0,0,0,0
846,1,0,0,1,1,1,0,0,0,0
1204,1,0,0,1,1,0,0,0,0,0
1206,1,1,1,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1613,1,0,0,1,1,0,0,0,0,0
1614,1,0,0,1,1,0,0,0,0,0
1615,1,0,0,1,1,0,0,0,0,0


In [37]:
# filter for the current component
accessneeds_df = accessneeds_df[accessneeds_df.index.isin(S)]
accessneeds_df

,5mt1,5mt2,5mt3,5mt4,5mt5,10mt1,10mt2,10mt3,10mt4,10mt5
ID_UG,,,,,,,,,,
1469,1,0,1,1,1,0,0,0,0,0
1470,1,0,1,1,1,0,0,0,0,0
942,1,0,1,1,1,0,0,0,0,0
945,1,0,0,1,1,0,0,0,0,0
947,1,1,0,1,1,0,0,0,0,0


In [38]:
# Modify column names to have t in T as column indices
accessneeds_df.columns = accessneeds_df.columns.str.split('t').str[-1].astype(int)

# Split the dataframe into two parts, one for 5m one for 10m
wide_accessneeds_df = accessneeds_df.iloc[:, -5:]  # Last 5 columns
accessneeds_df = accessneeds_df.iloc[:, :5]  # First 5 columns

print(accessneeds_df.head())
print(wide_accessneeds_df.head())

       1  2  3  4  5
ID_UG               
1469   1  0  1  1  1
1470   1  0  1  1  1
942    1  0  1  1  1
945    1  0  0  1  1
947    1  1  0  1  1
       1  2  3  4  5
ID_UG               
1469   0  0  0  0  0
1470   0  0  0  0  0
942    0  0  0  0  0
945    0  0  0  0  0
947    0  0  0  0  0


In [39]:
# doublecheck
T == accessneeds_df.columns

array([ True,  True,  True,  True,  True])

In [40]:
S

{942, 944, 945, 946, 947, 948, 1304, 1469, 1470}

In [41]:
accessneeds_df.index

Index([1469, 1470, 942, 945, 947], dtype='int64', name='ID_UG')

### Set needroad parameters, for all stands ID in S
$$needroad_{s,t}^\text{timber}$$

$$needroad_{s,t}^\text{wide}$$

In [42]:
# First for 'timber'
needroad_timber = {
    ('timber', s, t): accessneeds_df.loc[s, t] if s in accessneeds_df.index else 0
    for s in S
    for t in T
}

# Then for 'wide'
needroad_wide = {
    ('wide', s, t): wide_accessneeds_df.loc[s, t] if s in wide_accessneeds_df.index else 0
    for s in S
    for t in T
}


# Combine both dictionaries
needroad_w_s_t = {**needroad_timber, **needroad_wide}
needroad_w_s_t

{('timber', 942, 1): 1,
 ('timber', 942, 2): 0,
 ('timber', 942, 3): 1,
 ('timber', 942, 4): 1,
 ('timber', 942, 5): 1,
 ('timber', 944, 1): 0,
 ('timber', 944, 2): 0,
 ('timber', 944, 3): 0,
 ('timber', 944, 4): 0,
 ('timber', 944, 5): 0,
 ('timber', 945, 1): 1,
 ('timber', 945, 2): 0,
 ('timber', 945, 3): 0,
 ('timber', 945, 4): 1,
 ('timber', 945, 5): 1,
 ('timber', 946, 1): 0,
 ('timber', 946, 2): 0,
 ('timber', 946, 3): 0,
 ('timber', 946, 4): 0,
 ('timber', 946, 5): 0,
 ('timber', 947, 1): 1,
 ('timber', 947, 2): 1,
 ('timber', 947, 3): 0,
 ('timber', 947, 4): 1,
 ('timber', 947, 5): 1,
 ('timber', 948, 1): 0,
 ('timber', 948, 2): 0,
 ('timber', 948, 3): 0,
 ('timber', 948, 4): 0,
 ('timber', 948, 5): 0,
 ('timber', 1304, 1): 0,
 ('timber', 1304, 2): 0,
 ('timber', 1304, 3): 0,
 ('timber', 1304, 4): 0,
 ('timber', 1304, 5): 0,
 ('timber', 1469, 1): 1,
 ('timber', 1469, 2): 0,
 ('timber', 1469, 3): 1,
 ('timber', 1469, 4): 1,
 ('timber', 1469, 5): 1,
 ('timber', 1470, 1): 1,
 ('ti

### Calculate maxflow parameters
$$maxflow_t^\text{timber} = \sum_{s\in S}needroad_{s,t}^\text{timber}$$

In [43]:
w = 'timber'
maxflow_timber = {
    (w, t): sum(needroad_w_s_t[(w, s,t)] for s in S for t in T)
    for t in T
}
maxflow_timber

w = 'wide'
maxflow_wide = {
    (w, t): sum(needroad_w_s_t[(w, s,t)] for s in S for t in T)
    for t in T  # Loop over the time periods (columns)
}
maxflow_wide

{('wide', 1): 0,
 ('wide', 2): 0,
 ('wide', 3): 0,
 ('wide', 4): 0,
 ('wide', 5): 0}

$$maxflow_t^\text{wide} = \sum_{s\in S}needroad_{s,t}^\text{wide}$$

In [44]:
# Combine both dictionaries
maxflow_w_t = {**maxflow_timber, **maxflow_wide}
maxflow_w_t

{('timber', 1): 19,
 ('timber', 2): 19,
 ('timber', 3): 19,
 ('timber', 4): 19,
 ('timber', 5): 19,
 ('wide', 1): 0,
 ('wide', 2): 0,
 ('wide', 3): 0,
 ('wide', 4): 0,
 ('wide', 5): 0}

## Define binary decision variables 
for construction maintenance upgrade

### define

In [45]:
# Decision variables binary
C = { (e, w, t): f"C_{w}_{e}_t{t}" for e in E for w in W for t in (0,) + T }
M = { (e, w, t): f"M_{w}_{e}_t{t}" for e in E for w in W for t in (0,) + T }
U = { (e, t): f"U_{e}_t{t}" for e in E for t in (0,) + T }

### verify the created decision variables

In [46]:
print('len(E)*(len(T)+1) = ', len(E)*(len(T)+1))
print('C:', len(C), 'M:', len(M), 'U:', len(U))

len(E)*(len(T)+1) =  438
C: 876 M: 876 U: 438


In [47]:
C

{('node2node20', 'timber', 0): 'C_timber_node2node20_t0',
 ('node2node20', 'timber', 1): 'C_timber_node2node20_t1',
 ('node2node20', 'timber', 2): 'C_timber_node2node20_t2',
 ('node2node20', 'timber', 3): 'C_timber_node2node20_t3',
 ('node2node20', 'timber', 4): 'C_timber_node2node20_t4',
 ('node2node20', 'timber', 5): 'C_timber_node2node20_t5',
 ('node2node20', 'wide', 0): 'C_wide_node2node20_t0',
 ('node2node20', 'wide', 1): 'C_wide_node2node20_t1',
 ('node2node20', 'wide', 2): 'C_wide_node2node20_t2',
 ('node2node20', 'wide', 3): 'C_wide_node2node20_t3',
 ('node2node20', 'wide', 4): 'C_wide_node2node20_t4',
 ('node2node20', 'wide', 5): 'C_wide_node2node20_t5',
 ('node11node17', 'timber', 0): 'C_timber_node11node17_t0',
 ('node11node17', 'timber', 1): 'C_timber_node11node17_t1',
 ('node11node17', 'timber', 2): 'C_timber_node11node17_t2',
 ('node11node17', 'timber', 3): 'C_timber_node11node17_t3',
 ('node11node17', 'timber', 4): 'C_timber_node11node17_t4',
 ('node11node17', 'timber', 

## Define integer variables for flow

In [48]:
# Decision variables integer
flow = { (a, w, t): f"flow_{w}_{a}_t{t}" for a in A for w in W for t in T }
flow

{('node1node7', 'timber', 1): 'flow_timber_node1node7_t1',
 ('node1node7', 'timber', 2): 'flow_timber_node1node7_t2',
 ('node1node7', 'timber', 3): 'flow_timber_node1node7_t3',
 ('node1node7', 'timber', 4): 'flow_timber_node1node7_t4',
 ('node1node7', 'timber', 5): 'flow_timber_node1node7_t5',
 ('node1node7', 'wide', 1): 'flow_wide_node1node7_t1',
 ('node1node7', 'wide', 2): 'flow_wide_node1node7_t2',
 ('node1node7', 'wide', 3): 'flow_wide_node1node7_t3',
 ('node1node7', 'wide', 4): 'flow_wide_node1node7_t4',
 ('node1node7', 'wide', 5): 'flow_wide_node1node7_t5',
 ('node1node10', 'timber', 1): 'flow_timber_node1node10_t1',
 ('node1node10', 'timber', 2): 'flow_timber_node1node10_t2',
 ('node1node10', 'timber', 3): 'flow_timber_node1node10_t3',
 ('node1node10', 'timber', 4): 'flow_timber_node1node10_t4',
 ('node1node10', 'timber', 5): 'flow_timber_node1node10_t5',
 ('node1node10', 'wide', 1): 'flow_wide_node1node10_t1',
 ('node1node10', 'wide', 2): 'flow_wide_node1node10_t2',
 ('node1nod

## Build model

### Initialize model

In [49]:
# Initialize the CPLEX model
model = Cplex()
model.set_problem_type(Cplex.problem_type.LP)

### Add binary decision variables

In [50]:
model.variables.delete()  # deletes all variables

In [51]:
objective_coeffs = []
objective_terms = []

# Add C_vars
for (e, w, t), name in C.items():
    if t == 0:
        model.variables.add(names=[name], lb=[0], ub=[0], types=["B"])  # fixed to 0
    else:
        model.variables.add(names=[name], lb=[0], ub=[1], types=["B"])

    # prep list of variable names and cost coefficients for obj function
    objective_terms.append(C[(e, w, t)])
    objective_coeffs.append(CostC[(e, w)])

# Add M_vars
for (e, w, t), name in M.items():
    if t == 0:
        model.variables.add(names=[name], lb=[0], ub=[0], types=["B"])
    else:
        model.variables.add(names=[name], lb=[0], ub=[1], types=["B"])

    objective_terms.append(M[(e, w, t)])  
    objective_coeffs.append(CostM[(e, w)])

# Add U_vars
for (e, t), name in U.items():
    if t == 0:
        model.variables.add(names=[name], lb=[0], ub=[0], types=["B"])
    else:
        model.variables.add(names=[name], lb=[0], ub=[1], types=["B"])
        
    objective_terms.append(U[(e, t)])
    objective_coeffs.append(CostU[e])

#### verify
should be len(E) x (len (T)+1) x 5
because 5 different actions

In [52]:
n_action_variables = len(model.variables.get_names())
print(n_action_variables, 'action variables')
print('5*len(E)*len(T) =', len(E)*(len(T)+1)*5)
print(len(objective_coeffs), 'objective coeffs')

2190 action variables
5*len(E)*len(T) = 2190
2190 objective coeffs


In [53]:
# Get all variable names
var_names = model.variables.get_names()

# Loop through and get bounds or types
for name in var_names:
    lb = model.variables.get_lower_bounds([name])[0]
    ub = model.variables.get_upper_bounds([name])[0]
    var_type = model.variables.get_types([name])[0]
    print(f"{name}: lb={lb}, ub={ub}, type={var_type}")

    e = re.search(r"node[\w]+(?=_t\d)", name).group(0)
    if name[0] != "U":
        w = re.search(r"[CMU]_(\w+)(?=_node)", name).group(1)
        if name[0]=="C":
            cost_coeff = CostC[(e,w)]
        else:
            cost_coeff = CostM[(e,w)]
    else:
        cost_coeff = CostU[e]

C_timber_node2node20_t0: lb=0.0, ub=0.0, type=B
C_timber_node2node20_t1: lb=0.0, ub=1.0, type=B
C_timber_node2node20_t2: lb=0.0, ub=1.0, type=B
C_timber_node2node20_t3: lb=0.0, ub=1.0, type=B
C_timber_node2node20_t4: lb=0.0, ub=1.0, type=B
C_timber_node2node20_t5: lb=0.0, ub=1.0, type=B
C_wide_node2node20_t0: lb=0.0, ub=0.0, type=B
C_wide_node2node20_t1: lb=0.0, ub=1.0, type=B
C_wide_node2node20_t2: lb=0.0, ub=1.0, type=B
C_wide_node2node20_t3: lb=0.0, ub=1.0, type=B
C_wide_node2node20_t4: lb=0.0, ub=1.0, type=B
C_wide_node2node20_t5: lb=0.0, ub=1.0, type=B
C_timber_node11node17_t0: lb=0.0, ub=0.0, type=B
C_timber_node11node17_t1: lb=0.0, ub=1.0, type=B
C_timber_node11node17_t2: lb=0.0, ub=1.0, type=B
C_timber_node11node17_t3: lb=0.0, ub=1.0, type=B
C_timber_node11node17_t4: lb=0.0, ub=1.0, type=B
C_timber_node11node17_t5: lb=0.0, ub=1.0, type=B
C_wide_node11node17_t0: lb=0.0, ub=0.0, type=B
C_wide_node11node17_t1: lb=0.0, ub=1.0, type=B
C_wide_node11node17_t2: lb=0.0, ub=1.0, type=B
C

### Objective Function

In [54]:
obj_df = pd.DataFrame({
    'variable name': objective_terms,
    'Cost': objective_coeffs
})
obj_df

,variable name,Cost
0,C_timber_node2node20_t0,3120
1,C_timber_node2node20_t1,3120
2,C_timber_node2node20_t2,3120
3,C_timber_node2node20_t3,3120
4,C_timber_node2node20_t4,3120
...,...,...
2185,U_node14node19_t1,2002
2186,U_node14node19_t2,2002
2187,U_node14node19_t3,2002
2188,U_node14node19_t4,2002


In [55]:
len(objective_coeffs)

2190

In [56]:
print(f"Number of unique variables in objective: {len(set(objective_terms))}")

Number of unique variables in objective: 2190


In [57]:
objective_coeffs

[3120,
 3120,
 3120,
 3120,
 3120,
 3120,
 4677,
 4677,
 4677,
 4677,
 4677,
 4677,
 2998,
 2998,
 2998,
 2998,
 2998,
 2998,
 4495,
 4495,
 4495,
 4495,
 4495,
 4495,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2980,
 2980,
 2980,
 2980,
 2980,
 2980,
 4472,
 4472,
 4472,
 4472,
 4472,
 4472,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3041,
 3041,
 3041,
 3041,
 3041,
 3041,
 4562,
 4562,
 4562,
 4562,
 4562,
 4562,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 285,
 285,
 285,
 285,
 285,
 285,
 428,
 428,
 428,
 428,
 428,
 428,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1845,
 1845,
 1845,
 1845,
 1845,
 1845,
 2770,
 2770,
 2770,
 2770,
 2770,
 2770,
 2919,
 2919,
 2919,
 2919,
 2919,
 2919,
 4378,
 4378,
 4378,
 4378,
 4378,
 4378,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [58]:
# Set the objective to minimize
model.objective.set_sense(model.objective.sense.minimize)

# Assign the linear objective function
# Each variable in objective_terms is multiplied by the corresponding value in objective_coeffs
model.objective.set_linear(list(zip(objective_terms, objective_coeffs)))

#### verify objective function

In [59]:
objective_expr = model.objective.get_linear()

for i in range(len(objective_expr)):
    var_name = model.variables.get_names(i)
    coeff = objective_expr[i]
    print(f"{coeff} * {var_name}")

3120.0 * C_timber_node2node20_t0
3120.0 * C_timber_node2node20_t1
3120.0 * C_timber_node2node20_t2
3120.0 * C_timber_node2node20_t3
3120.0 * C_timber_node2node20_t4
3120.0 * C_timber_node2node20_t5
4677.0 * C_wide_node2node20_t0
4677.0 * C_wide_node2node20_t1
4677.0 * C_wide_node2node20_t2
4677.0 * C_wide_node2node20_t3
4677.0 * C_wide_node2node20_t4
4677.0 * C_wide_node2node20_t5
2998.0 * C_timber_node11node17_t0
2998.0 * C_timber_node11node17_t1
2998.0 * C_timber_node11node17_t2
2998.0 * C_timber_node11node17_t3
2998.0 * C_timber_node11node17_t4
2998.0 * C_timber_node11node17_t5
4495.0 * C_wide_node11node17_t0
4495.0 * C_wide_node11node17_t1
4495.0 * C_wide_node11node17_t2
4495.0 * C_wide_node11node17_t3
4495.0 * C_wide_node11node17_t4
4495.0 * C_wide_node11node17_t5
0.0 * C_timber_node12node21_t0
0.0 * C_timber_node12node21_t1
0.0 * C_timber_node12node21_t2
0.0 * C_timber_node12node21_t3
0.0 * C_timber_node12node21_t4
0.0 * C_timber_node12node21_t5
0.0 * C_wide_node12node21_t0
0.0 *

### Add flow variables

In [60]:
len(S)

9

In [61]:
# Add variables to the model
for (e,w,t), name in flow.items():
    #upperbound = len(S)
    model.variables.add(names=[name], lb=[0], ub=[9], types=["I"])

##### verify
should be 2 x len(E) x len(T) variables

In [62]:
# verify 
n_variables = len(model.variables.get_names())
print(n_variables, 'variables')

n_flow_variables = n_variables - n_action_variables
print(n_flow_variables, 'flow variables')

print('len(A)*len(T)*len(W) =', len(A)*len(T)*len(W))

3650 variables
1460 flow variables
len(A)*len(T)*len(W) = 1460


In [63]:
var_names = model.variables.get_names()[-n_flow_variables:]

# Loop through and get bounds or types
for name in var_names:
    lb = model.variables.get_lower_bounds([name])[0]
    ub = model.variables.get_upper_bounds([name])[0]
    var_type = model.variables.get_types([name])[0]
    print(f"{name}: lb={lb}, ub={ub}, type={var_type}")

flow_timber_node1node7_t1: lb=0.0, ub=9.0, type=I
flow_timber_node1node7_t2: lb=0.0, ub=9.0, type=I
flow_timber_node1node7_t3: lb=0.0, ub=9.0, type=I
flow_timber_node1node7_t4: lb=0.0, ub=9.0, type=I
flow_timber_node1node7_t5: lb=0.0, ub=9.0, type=I
flow_wide_node1node7_t1: lb=0.0, ub=9.0, type=I
flow_wide_node1node7_t2: lb=0.0, ub=9.0, type=I
flow_wide_node1node7_t3: lb=0.0, ub=9.0, type=I
flow_wide_node1node7_t4: lb=0.0, ub=9.0, type=I
flow_wide_node1node7_t5: lb=0.0, ub=9.0, type=I
flow_timber_node1node10_t1: lb=0.0, ub=9.0, type=I
flow_timber_node1node10_t2: lb=0.0, ub=9.0, type=I
flow_timber_node1node10_t3: lb=0.0, ub=9.0, type=I
flow_timber_node1node10_t4: lb=0.0, ub=9.0, type=I
flow_timber_node1node10_t5: lb=0.0, ub=9.0, type=I
flow_wide_node1node10_t1: lb=0.0, ub=9.0, type=I
flow_wide_node1node10_t2: lb=0.0, ub=9.0, type=I
flow_wide_node1node10_t3: lb=0.0, ub=9.0, type=I
flow_wide_node1node10_t4: lb=0.0, ub=9.0, type=I
flow_wide_node1node10_t5: lb=0.0, ub=9.0, type=I
flow_timbe

## Constraints

### Initialize

In [64]:
# init
model.linear_constraints.delete()

### Flow (wide) enforces road existence (wide)

$$\forall t \in T \quad \forall e\in E: \qquad
flow_{(u,v),t}^{\text{wide}} + flow_{(v,u),t}^{\text{wide}}  \leq maxflow_t^{\text{wide}} \big(C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big)$$
$$\Leftrightarrow$$
$$flow_{(u,v),t}^{\text{wide}} + flow_{(v,u),t}^{\text{wide}} - maxflow_t^{\text{wide}} \big(C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big)  \leq 0$$
$$\Leftrightarrow$$
$$
flow_{(u,v),t}^{\text{wide}} + flow_{(v,u),t}^{\text{wide}} - maxflow_t^{\text{wide}}C _{e,t}^{\text{wide}} - maxflow_t^{\text{wide}}M_{e,t}^{\text{wide}} - maxflow_t^{\text{wide}}U_{e,t} \leq 0\\
$$

In [65]:
# add wide flow enforces wide roads
w = 'wide'
for t in T:
    for a_fw in A_fw:
        a_bw = fw_to_bw_map[a_fw]
        e = a_fw
        try:
            # Define the linear expression components for better clarity
            expr_vars = [flow[(a_fw, w, t)], flow[(a_bw, w, t)], C[(e, w, t)], M[(e, w, t)], U[(e, t)]]
            coeff = int(maxflow_w_t[(w,t)])
            expr_coeffs = [1, 1, -coeff , -coeff, -coeff]
            
            # Add the constraint to the model
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["L"],  # 'L' indicates "less than or equal to"
                rhs=[0],  # Right-hand side of the constraint
                names=[f"Constraint_flow_enforcing_road_{w}_{e}_{t}"]
            )
            
            # Print what was added to the model (variables and coefficients)
            print(f"Added constraint flow_enforcing_road_{w}_{e}_{t}")
            print(f"  Variables: {expr_vars}")
            print(f"  Coefficients: {expr_coeffs}")

        except KeyError as ke:
            print(f"KeyError: {ke} - This indicates a missing variable for (e={e}, t={t})")

Added constraint flow_enforcing_road_wide_node2node20_1
  Variables: ['flow_wide_node2node20_t1', 'flow_wide_node20node2_t1', 'C_wide_node2node20_t1', 'M_wide_node2node20_t1', 'U_node2node20_t1']
  Coefficients: [1, 1, 0, 0, 0]
Added constraint flow_enforcing_road_wide_node11node17_1
  Variables: ['flow_wide_node11node17_t1', 'flow_wide_node17node11_t1', 'C_wide_node11node17_t1', 'M_wide_node11node17_t1', 'U_node11node17_t1']
  Coefficients: [1, 1, 0, 0, 0]
Added constraint flow_enforcing_road_wide_node12node21_1
  Variables: ['flow_wide_node12node21_t1', 'flow_wide_node21node12_t1', 'C_wide_node12node21_t1', 'M_wide_node12node21_t1', 'U_node12node21_t1']
  Coefficients: [1, 1, 0, 0, 0]
Added constraint flow_enforcing_road_wide_node12node22_1
  Variables: ['flow_wide_node12node22_t1', 'flow_wide_node22node12_t1', 'C_wide_node12node22_t1', 'M_wide_node12node22_t1', 'U_node12node22_t1']
  Coefficients: [1, 1, 0, 0, 0]
Added constraint flow_enforcing_road_wide_node3node8_1
  Variables: ['

##### verify
should be 5 x len(E) constraints

In [66]:
print(f"Total number of constraints in the model: {model.linear_constraints.get_num()}")
n_constraints_group1 = model.linear_constraints.get_num()
print(f"Total number of constraints in this group:", n_constraints_group1)

Total number of constraints in the model: 365
Total number of constraints in this group: 365


### Flow (timber) enforcing road existence

$$
flow_{(u,v),t}^{\text{timber}} + flow_{(v,u),t}^{\text{timber}}\leq maxflow_{t}^{\text{timber}} \big(C_{e,t}^{\text{timber}}+ M_{e,t}^{\text{timber}} + C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big)$$
$$
\Leftrightarrow$$
$$flow_{(u,v),t}^{\text{timber}} + flow_{(v,u),t}^{\text{timber}}- maxflow_{t}^{\text{timber}} \big(C_{e,t}^{\text{timber}}+ M_{e,t}^{\text{timber}} + C _{e,t}^{\text{wide}} + M_{e,t}^{\text{wide}} +  U_{e,t}\big) \leq 0$$
$$\Leftrightarrow$$
$$flow_{(u,v),t}^{\text{timber}} + flow_{(v,u),t}^{\text{timber}}- maxflow_{t}^{\text{timber}} C_{e,t}^{\text{timber}}-maxflow_{t}^{\text{timber}}  M_{e,t}^{\text{timber}} -maxflow_{t}^{\text{timber}}  C _{e,t}^{\text{wide}} -maxflow_{t}^{\text{timber}}  M_{e,t}^{\text{wide}} -maxflow_{t}^{\text{timber}}  U_{e,t} \leq 0$$

In [67]:
# add timber flow enforces at least timber roads
for t in T:
    for a_fw in E:
        a_bw = fw_to_bw_map[a_fw]
        e = a_fw
        try:
            # Define the linear expression components for better clarity
            expr_vars = [flow[(a_fw, 'timber', t)], flow[(a_bw, 'timber', t)], C[(e, 'timber', t)], M[(e, 'timber', t)], C[(e, 'wide', t)], M[(e, 'wide', t)],U[(e, t)]]
            coeff = int(maxflow_w_t[('timber',t)])
            expr_coeffs = [1, 1, -coeff , -coeff, -coeff, -coeff, -coeff]
            
            # Add the constraint to the model
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["L"],  # 'L' indicates "less than or equal to"
                rhs=[0],  # Right-hand side of the constraint
                names=[f"Constraint_flow_enforcing_road_{w}_{e}_{t}"]
            )
            
            # Print what was added to the model (variables and coefficients)
            print(f"Added constraint flow_enforcing_road_{w}_{e}_{t}")
            print(f"  Variables: {expr_vars}")
            print(f"  Coefficients: {expr_coeffs}")

        except KeyError as ke:
            print(f"KeyError: {ke} - This indicates a missing variable for (e={e}, t={t})")

Added constraint flow_enforcing_road_wide_node2node20_1
  Variables: ['flow_timber_node2node20_t1', 'flow_timber_node20node2_t1', 'C_timber_node2node20_t1', 'M_timber_node2node20_t1', 'C_wide_node2node20_t1', 'M_wide_node2node20_t1', 'U_node2node20_t1']
  Coefficients: [1, 1, -19, -19, -19, -19, -19]
Added constraint flow_enforcing_road_wide_node11node17_1
  Variables: ['flow_timber_node11node17_t1', 'flow_timber_node17node11_t1', 'C_timber_node11node17_t1', 'M_timber_node11node17_t1', 'C_wide_node11node17_t1', 'M_wide_node11node17_t1', 'U_node11node17_t1']
  Coefficients: [1, 1, -19, -19, -19, -19, -19]
Added constraint flow_enforcing_road_wide_node12node21_1
  Variables: ['flow_timber_node12node21_t1', 'flow_timber_node21node12_t1', 'C_timber_node12node21_t1', 'M_timber_node12node21_t1', 'C_wide_node12node21_t1', 'M_wide_node12node21_t1', 'U_node12node21_t1']
  Coefficients: [1, 1, -19, -19, -19, -19, -19]
Added constraint flow_enforcing_road_wide_node12node22_1
  Variables: ['flow_t

##### verify
should be 5 x len(E) constraints

In [68]:
print(f"Total number of constraints in the model: {model.linear_constraints.get_num()}")
n_constraints_group2 = model.linear_constraints.get_num() - n_constraints_group1
print(f"Total number of constraints in this group:", n_constraints_group2)
print(len(E))

Total number of constraints in the model: 730
Total number of constraints in this group: 365
73


### Flow conservation

$$\forall t \in T \quad \forall v \in V_{\text{transit}}\quad \forall w \in W$$
$
\begin{align}
       \sum_{u \in N(v)} flow_{(u,v),t}^{w} &= \sum_{u \in N(v)} flow_{(v,u),t}^{w}\\
       &\Leftrightarrow
       \\
       \sum_{u \in N(v)} flow_{(u,v),t}^{w} - \sum_{u \in N(v)} flow_{(v,u),t}^{w}&=0
\end{align}
$

In [69]:
# add flow conservation constraints
for w in W:
    for t in T:
        for tnode, in_arcs in ingoing_per_transitnode.items():
            out_arcs = outgoing_per_transitnode[tnode]
            try:
                # Define the linear expression components 
                expr_vars = [flow[(a_in, w, t)] for a_in in in_arcs] + [flow[(a_out, w, t)] for a_out in out_arcs]
                expr_coeffs = [1] * len(in_arcs) + [-1] * len(out_arcs)

                # Add the constraint to the model
                model.linear_constraints.add(
                    lin_expr=[[expr_vars, expr_coeffs]],
                    senses=["E"],  # 'E' indicates "equal to"
                    rhs=[0],  # Right-hand side of the constraint
                    names=[f"Constraint_flow_conservation_{w}_{t}_{tnode}"]
                )
                
                # Print what was added to the model (variables and coefficients)
                print(f"Added constraint flow_conservation_{w}_{t}_{tnode}")
                print(f"  Variables: {expr_vars}")
                print(f"  Coefficients: {expr_coeffs}")

            except KeyError as ke:
                print(f"KeyError: {ke} - This indicates a missing variable for (e={tnode}, w={w}, t={t})")

Added constraint flow_conservation_timber_1_node3
  Variables: ['flow_timber_node24node3_t1', 'flow_timber_node29node3_t1', 'flow_timber_node5node3_t1', 'flow_timber_node11node3_t1', 'flow_timber_node8node3_t1', 'flow_timber_node27node3_t1', 'flow_timber_node3node11_t1', 'flow_timber_node3node29_t1', 'flow_timber_node3node8_t1', 'flow_timber_node3node27_t1', 'flow_timber_node3node5_t1', 'flow_timber_node3node24_t1']
  Coefficients: [1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1]
Added constraint flow_conservation_timber_1_node16
  Variables: ['flow_timber_node25node16_t1', 'flow_timber_node18node16_t1', 'flow_timber_node15node16_t1', 'flow_timber_node22node16_t1', 'flow_timber_node12node16_t1', 'flow_timber_node21node16_t1', 'flow_timber_node16node22_t1', 'flow_timber_node16node21_t1', 'flow_timber_node16node25_t1', 'flow_timber_node16node12_t1', 'flow_timber_node16node15_t1', 'flow_timber_node16node18_t1']
  Coefficients: [1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1]
Added constraint flow_con

#### verify
should be 2 x len(V_transit) x 5

In [70]:
print(f"Total number of constraints in the model: {model.linear_constraints.get_num()}")
n_constraints_group3 = model.linear_constraints.get_num() - n_constraints_group1 - n_constraints_group2
print(f"Total number of constraints in this group:", n_constraints_group3)
print(f'Transit nodes {len(V_transit)}')

Total number of constraints in the model: 890
Total number of constraints in this group: 160
Transit nodes 16


### Outflow if and only if stand needs road

$\begin{align}
\forall t \in T\quad\forall s\in V_{source}\quad\forall w\in W:\quad\\
   \sum_{v\in B(s)}flow_{(s,v),t}^{w} &= need\_road_{s,t}^{w}
\end{align}
$

In [71]:
# add outflow only from stands that need road
for w in W:
    for t in T:
        for s, out_arcs in source_arcs_outgoing.items():
            print(s)
            try:
                # Define the linear expression components for better clarity
                expr_vars = [flow[(e, w, t)] for e in out_arcs]
                expr_coeffs = [1] * len(out_arcs)
                rhs = int(needroad_w_s_t[(w,s,t)])
                
                # Add the constraint to the model
                model.linear_constraints.add(
                    lin_expr=[[expr_vars, expr_coeffs]],
                    senses=["E"],  # 'equal to"
                    rhs=[rhs],  # Right-hand side of the constraint
                    names=[f"Constraint_outflow_iff_needroad_{w}_{s}_t{t}"]
                )
                
                # Print what was added to the model (variables and coefficients)
                print(f"Added constraint outflow_iff_needroad_{w}_{s}_t{t}")
                print(f"  Variables: {expr_vars}")
                print(f"  Coefficients: {expr_coeffs}")
                print(f"  RHS: {rhs}")

            except KeyError as ke:
                print(f"KeyError: {ke} - This indicates a missing variable for (s={s}, w={w}, t={t})")

1469
Added constraint outflow_iff_needroad_timber_1469_t1
  Variables: ['flow_timber_node21node12_t1', 'flow_timber_node21node16_t1', 'flow_timber_node21node18_t1', 'flow_timber_node21node17_t1']
  Coefficients: [1, 1, 1, 1]
  RHS: 1
1470
Added constraint outflow_iff_needroad_timber_1470_t1
  Variables: ['flow_timber_node22node16_t1', 'flow_timber_node22node12_t1', 'flow_timber_node22node15_t1']
  Coefficients: [1, 1, 1]
  RHS: 1
942
Added constraint outflow_iff_needroad_timber_942_t1
  Variables: ['flow_timber_node23node17_t1', 'flow_timber_node23node6_t1', 'flow_timber_node23node15_t1', 'flow_timber_node23node12_t1', 'flow_timber_node23node11_t1', 'flow_timber_node23node9_t1']
  Coefficients: [1, 1, 1, 1, 1, 1]
  RHS: 1
944
Added constraint outflow_iff_needroad_timber_944_t1
  Variables: ['flow_timber_node24node5_t1', 'flow_timber_node24node3_t1', 'flow_timber_node24node4_t1', 'flow_timber_node24node14_t1', 'flow_timber_node24node8_t1', 'flow_timber_node24node19_t1']
  Coefficients: 

##### verify
should be len(V_source) x 5 actions x 2

In [72]:
print(f"Total number of constraints in the model: {model.linear_constraints.get_num()}")
n_constraints_group4 = model.linear_constraints.get_num() - n_constraints_group3 - n_constraints_group2 - n_constraints_group1
print(f"Total number of constraints in this group:", n_constraints_group4)
print('source nodes', len(V_source))

Total number of constraints in the model: 980
Total number of constraints in this group: 90
source nodes 9


### No Inflow to source nodes

$\begin{align}
\forall t \in T\quad\forall s\in V_{source}:\quad
    \sum_{u\in B(s)} \Big(flow_{(u,s),t}^{\text{wide}} + flow_{(u,s),t}^{\text{timber}}\Big) = 0
\end{align}
$

In [73]:
# don't allow inflow
for t in T:
    for s, in_arcs in source_arcs_ingoing.items():
        print(s)
        try:
            # Define the linear expression components for better clarity
            expr_vars = [flow[(a, 'timber', t)] for a in in_arcs] + [flow[(a, 'wide', t)] for a in in_arcs]
            expr_coeffs = [1] * len(in_arcs) + [1] * len(in_arcs)

            # Add the constraint to the model
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["E"],  # 'E' indicates "equal to"
                rhs=[0],  # Right-hand side of the constraint
                names=[f"Constraint_no_inflow_source_{s}_{t}"]
            )
            
            # Print what was added to the model (variables and coefficients)
            print(f"Added constraint no_inflow_source_{s}_{t}")
            print(f"  Variables: {expr_vars}")
            print(f"  Coefficients: {expr_coeffs}")

        except KeyError as ke:
            print(f"KeyError: {ke} - This indicates a missing variable for (s={s}, w={w}, t={t})")

1469
Added constraint no_inflow_source_1469_1
  Variables: ['flow_timber_node18node21_t1', 'flow_timber_node16node21_t1', 'flow_timber_node17node21_t1', 'flow_timber_node12node21_t1', 'flow_wide_node18node21_t1', 'flow_wide_node16node21_t1', 'flow_wide_node17node21_t1', 'flow_wide_node12node21_t1']
  Coefficients: [1, 1, 1, 1, 1, 1, 1, 1]
1470
Added constraint no_inflow_source_1470_1
  Variables: ['flow_timber_node15node22_t1', 'flow_timber_node12node22_t1', 'flow_timber_node16node22_t1', 'flow_wide_node15node22_t1', 'flow_wide_node12node22_t1', 'flow_wide_node16node22_t1']
  Coefficients: [1, 1, 1, 1, 1, 1]
942
Added constraint no_inflow_source_942_1
  Variables: ['flow_timber_node6node23_t1', 'flow_timber_node9node23_t1', 'flow_timber_node12node23_t1', 'flow_timber_node11node23_t1', 'flow_timber_node15node23_t1', 'flow_timber_node17node23_t1', 'flow_wide_node6node23_t1', 'flow_wide_node9node23_t1', 'flow_wide_node12node23_t1', 'flow_wide_node11node23_t1', 'flow_wide_node15node23_t1',

#### verify
should be len(V_source)*5

In [74]:
print(f"Total number of constraints in the model: {model.linear_constraints.get_num()}")
n_constraints_group5 = model.linear_constraints.get_num() - n_constraints_group4 - n_constraints_group3 - n_constraints_group2 - n_constraints_group1
print(f"Total number of constraints in this group:", n_constraints_group5)
print('source nodes', len(V_source))

Total number of constraints in the model: 1025
Total number of constraints in this group: 45
source nodes 9


### Maintenance & Upgrade Constraints for timber roads

$$M_{e,t}^{\text{timber}} + U_{e,t} \leq C_{e,t-1}^{\text{timber}} + M_{e,t-1}^{\text{timber}}$$
$$\Leftrightarrow$$
$$M_{e,t}^{\text{timber}} + U_{e,t} - C_{e,t-1}^{\text{timber}} - M_{e,t-1}^{\text{timber}} \leq 0$$


In [75]:
# Add Maintenance or Upgrade constraint for 5-meter roads
w = 'timber'
for t in T:
    for e in E:
        try:
            # Define the linear expression components for better clarity
            expr_vars = [M[(e, w, t)], U[(e, t)], C[(e, w, t-1)], M[(e, w, t-1)]]
            expr_coeffs = [1, 1, -1, -1]
            
            # Add the constraint to the model
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["L"],  # 'L' indicates "less than or equal to"
                rhs=[0],  # Right-hand side of the constraint
                names=[f"Constraint_maintain_upgrade_{w}_{e}_{t}"]
            )
            
            # Print what was added to the model (variables and coefficients)
            print(f"Added constraint with name: maintain_upgrade_{w}_{e}_{t}")
            print(f"  Variables: {expr_vars}")
            print(f"  Coefficients: {expr_coeffs}")

        except KeyError as ke:
            print(f"KeyError: {ke} - This indicates a missing variable for (e={e}, t={t})")

Added constraint with name: maintain_upgrade_timber_node2node20_1
  Variables: ['M_timber_node2node20_t1', 'U_node2node20_t1', 'C_timber_node2node20_t0', 'M_timber_node2node20_t0']
  Coefficients: [1, 1, -1, -1]
Added constraint with name: maintain_upgrade_timber_node11node17_1
  Variables: ['M_timber_node11node17_t1', 'U_node11node17_t1', 'C_timber_node11node17_t0', 'M_timber_node11node17_t0']
  Coefficients: [1, 1, -1, -1]
Added constraint with name: maintain_upgrade_timber_node12node21_1
  Variables: ['M_timber_node12node21_t1', 'U_node12node21_t1', 'C_timber_node12node21_t0', 'M_timber_node12node21_t0']
  Coefficients: [1, 1, -1, -1]
Added constraint with name: maintain_upgrade_timber_node12node22_1
  Variables: ['M_timber_node12node22_t1', 'U_node12node22_t1', 'C_timber_node12node22_t0', 'M_timber_node12node22_t0']
  Coefficients: [1, 1, -1, -1]
Added constraint with name: maintain_upgrade_timber_node3node8_1
  Variables: ['M_timber_node3node8_t1', 'U_node3node8_t1', 'C_timber_nod

#### verify
should be 5x len(E)

In [76]:
print(f"Total number of constraints in the model: {model.linear_constraints.get_num()}")
n_constraints_group6 = model.linear_constraints.get_num() - n_constraints_group5 - n_constraints_group4 - n_constraints_group3 - n_constraints_group2 - n_constraints_group1
print(f"Total number of constraints in this group:", n_constraints_group6)

Total number of constraints in the model: 1390
Total number of constraints in this group: 365


### Maintenance/ Upgrade constraints for wide roads

$\begin{align}
M_{e,t}^{wide} &\leq C_{e,t-1}^{wide} + M_{e,t-1}^{wide} + U_{e,t-1}\\
\\
&\Leftrightarrow\\\
\\
M_{e,t}^{wide} - C_{e,t-1}^{wide} - M_{e,t-1}^{wide} - U_{e,t-1} &\leq 0
\end{align}
$

In [77]:
# Add Maintenance or Upgrade constraint for wide roads
w = 'wide'
for t in T:
    for e in E:
        try:
            # Define the linear expression components for better clarity
            expr_vars = [M[(e, w, t)], C[(e, w, t-1)], M[(e, w, t-1)], U[(e, t-1)]]
            expr_coeffs = [1, -1, -1, -1]
            print(expr_vars)
            print(expr_coeffs)
            
            # Add the constraint to the model
            model.linear_constraints.add(
                lin_expr=[[expr_vars, expr_coeffs]],
                senses=["L"],  # 'L' indicates "less than or equal to"
                rhs=[0],  # Right-hand side of the constraint
                names=[f"Constraint_maintain_upgrade_{w}_{e}_{t}"]
            )
            
            # Print what was added to the model (variables and coefficients)
            print(f"Added constraint with name: maintain_upgrade_{w}_{e}_{t}")
            print(f"  Variables: {expr_vars}")
            print(f"  Coefficients: {expr_coeffs}")

        except KeyError as ke:
            print(f"KeyError: {ke} - This indicates a missing variable for (e={e}, t={t})")

['M_wide_node2node20_t1', 'C_wide_node2node20_t0', 'M_wide_node2node20_t0', 'U_node2node20_t0']
[1, -1, -1, -1]
Added constraint with name: maintain_upgrade_wide_node2node20_1
  Variables: ['M_wide_node2node20_t1', 'C_wide_node2node20_t0', 'M_wide_node2node20_t0', 'U_node2node20_t0']
  Coefficients: [1, -1, -1, -1]
['M_wide_node11node17_t1', 'C_wide_node11node17_t0', 'M_wide_node11node17_t0', 'U_node11node17_t0']
[1, -1, -1, -1]
Added constraint with name: maintain_upgrade_wide_node11node17_1
  Variables: ['M_wide_node11node17_t1', 'C_wide_node11node17_t0', 'M_wide_node11node17_t0', 'U_node11node17_t0']
  Coefficients: [1, -1, -1, -1]
['M_wide_node12node21_t1', 'C_wide_node12node21_t0', 'M_wide_node12node21_t0', 'U_node12node21_t0']
[1, -1, -1, -1]
Added constraint with name: maintain_upgrade_wide_node12node21_1
  Variables: ['M_wide_node12node21_t1', 'C_wide_node12node21_t0', 'M_wide_node12node21_t0', 'U_node12node21_t0']
  Coefficients: [1, -1, -1, -1]
['M_wide_node12node22_t1', 'C_w

#### verify

In [78]:
print(f"Total number of constraints in the model: {model.linear_constraints.get_num()}")
n_constraints_group7 = model.linear_constraints.get_num() - n_constraints_group6 - n_constraints_group5 - n_constraints_group4 - n_constraints_group3 - n_constraints_group2 - n_constraints_group1
print(f"Total number of constraints in this group:", n_constraints_group7)

Total number of constraints in the model: 1755
Total number of constraints in this group: 365


### verify

In [79]:
for i in range(model.linear_constraints.get_num()):
    indices = model.linear_constraints.get_rows(i).ind
    coefs = model.linear_constraints.get_rows(i).val
    name = model.linear_constraints.get_names(i)
    rhs   = model.linear_constraints.get_rhs(i)
    sense = model.linear_constraints.get_senses(i)

    vars_in_constraint = [model.variables.get_names(idx) for idx in indices]
    terms = [f"{coef}*{var}" for coef, var in zip(coefs, vars_in_constraint)]
    expression = " + ".join(terms)
    
    print(f"{name or 'unnamed'}: {expression} {sense} {rhs}")

Constraint_flow_enforcing_road_wide_node2node20_1: 1.0*flow_wide_node2node20_t1 + 1.0*flow_wide_node20node2_t1 L 0.0
Constraint_flow_enforcing_road_wide_node11node17_1: 1.0*flow_wide_node11node17_t1 + 1.0*flow_wide_node17node11_t1 L 0.0
Constraint_flow_enforcing_road_wide_node12node21_1: 1.0*flow_wide_node12node21_t1 + 1.0*flow_wide_node21node12_t1 L 0.0
Constraint_flow_enforcing_road_wide_node12node22_1: 1.0*flow_wide_node12node22_t1 + 1.0*flow_wide_node22node12_t1 L 0.0
Constraint_flow_enforcing_road_wide_node3node8_1: 1.0*flow_wide_node3node8_t1 + 1.0*flow_wide_node8node3_t1 L 0.0
Constraint_flow_enforcing_road_wide_node17node27_1: 1.0*flow_wide_node27node17_t1 + 1.0*flow_wide_node17node27_t1 L 0.0
Constraint_flow_enforcing_road_wide_node3node24_1: 1.0*flow_wide_node3node24_t1 + 1.0*flow_wide_node24node3_t1 L 0.0
Constraint_flow_enforcing_road_wide_node2node14_1: 1.0*flow_wide_node2node14_t1 + 1.0*flow_wide_node14node2_t1 L 0.0
Constraint_flow_enforcing_road_wide_node7node25_1: 1.0*

## Solve

In [80]:
# Solve the model
model.solve()
solution = model.solution.get_values()
print(f"Optimal values: {solution}")

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 1314 rows and 2764 columns.
MIP Presolve modified 624 coefficients.
Aggregator did 4 substitutions.
Reduced MIP has 437 rows, 882 columns, and 2192 nonzeros.
Reduced MIP has 622 binaries, 260 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (3.85 ticks)
Found incumbent of value 689871.000000 after 0.03 sec. (7.24 ticks)
Probing time = 0.00 sec. (0.17 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 437 rows, 882 columns, and 2192 nonzeros.
Reduced MIP has 622 binaries, 260 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.48 ticks)
Probing time = 0.00 sec. (0.17 ticks)
Clique table members: 71.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.02 sec. (3.03 ticks)

In [81]:
# Get and print the solution
solution = model.solution
if solution.is_primal_feasible():
    print(f"Objective value: {solution.get_objective_value()}")
    for var_name in model.variables.get_names():
        print(f"{var_name} = {solution.get_values(var_name)}")
else:
    print("No feasible solution found.")

Objective value: 16745.0
C_timber_node2node20_t0 = 0.0
C_timber_node2node20_t1 = 0.0
C_timber_node2node20_t2 = 0.0
C_timber_node2node20_t3 = 0.0
C_timber_node2node20_t4 = 0.0
C_timber_node2node20_t5 = 0.0
C_wide_node2node20_t0 = 0.0
C_wide_node2node20_t1 = 0.0
C_wide_node2node20_t2 = 0.0
C_wide_node2node20_t3 = 0.0
C_wide_node2node20_t4 = 0.0
C_wide_node2node20_t5 = 0.0
C_timber_node11node17_t0 = 0.0
C_timber_node11node17_t1 = 1.0
C_timber_node11node17_t2 = 0.0
C_timber_node11node17_t3 = 0.0
C_timber_node11node17_t4 = 0.0
C_timber_node11node17_t5 = 0.0
C_wide_node11node17_t0 = 0.0
C_wide_node11node17_t1 = 0.0
C_wide_node11node17_t2 = 0.0
C_wide_node11node17_t3 = 0.0
C_wide_node11node17_t4 = 0.0
C_wide_node11node17_t5 = 0.0
C_timber_node12node21_t0 = 0.0
C_timber_node12node21_t1 = 1.0
C_timber_node12node21_t2 = 1.0
C_timber_node12node21_t3 = 1.0
C_timber_node12node21_t4 = 1.0
C_timber_node12node21_t5 = 1.0
C_wide_node12node21_t0 = 0.0
C_wide_node12node21_t1 = 1.0
C_wide_node12node21_t2 

## 💾 store the results

In [173]:
# Assume `model` is your CPLEX model and you've already solved it
solution = model.solution  # Get the solution object

# Get the variable names and their values
variable_names = model.variables.get_names()  # List of variable names
variable_values = solution.get_values()  # List of corresponding variable values

# Create a DataFrame
solution_df = pd.DataFrame({
    'Variable': variable_names,
    'Value': variable_values
})

# Optionally, if you want to filter or process, you can do so here
solution_df

,Variable,Value
0,C_timber_node2node20_t0,0.0
1,C_timber_node2node20_t1,0.0
2,C_timber_node2node20_t2,0.0
3,C_timber_node2node20_t3,0.0
4,C_timber_node2node20_t4,0.0
...,...,...
3645,flow_wide_node22node16_t1,0.0
3646,flow_wide_node22node16_t2,0.0
3647,flow_wide_node22node16_t3,0.0
3648,flow_wide_node22node16_t4,0.0


In [174]:
# Create the 't' column by extracting the number after the last underscore
solution_df['period'] = solution_df['Variable'].str.extract(r'_t(\d+)$')

# Create the 'action' column based on the prefix
def determine_action(variable):
    if variable.startswith('C'):
        return 'Construct'
    elif variable.startswith('M'):
        return 'Maintain'
    elif variable.startswith('U'):
        return 'Upgrade'
    elif variable.startswith('flow'):
        return 'Flow'
    return 'Unknown'

solution_df['action'] = solution_df['Variable'].apply(determine_action)

# Create the 'roadtype' column by extracting 'timber' or 'wide'
solution_df['type'] = solution_df['Variable'].str.extract(r'_(timber|wide)_')

# Create the 'edge' column by extracting the part like 'node8node14'
solution_df['edge'] = solution_df['Variable'].str.extract(r'_(node\d+node\d+)')

solution_df

,Variable,Value,period,action,type,edge
0,C_timber_node2node20_t0,0.0,0,Construct,timber,node2node20
1,C_timber_node2node20_t1,0.0,1,Construct,timber,node2node20
2,C_timber_node2node20_t2,0.0,2,Construct,timber,node2node20
3,C_timber_node2node20_t3,0.0,3,Construct,timber,node2node20
4,C_timber_node2node20_t4,0.0,4,Construct,timber,node2node20
...,...,...,...,...,...,...
3645,flow_wide_node22node16_t1,0.0,1,Flow,wide,node22node16
3646,flow_wide_node22node16_t2,0.0,2,Flow,wide,node22node16
3647,flow_wide_node22node16_t3,0.0,3,Flow,wide,node22node16
3648,flow_wide_node22node16_t4,0.0,4,Flow,wide,node22node16


In [122]:
V_exit

['node2', 'node4', 'node19', 'node20']

In [175]:
solution_df[(solution_df['Value'] != 0) & (solution_df['period'] == '1') & (solution_df.action=='Flow')]

,Variable,Value,period,action,type,edge
2450,flow_timber_node25node18_t1,1.0,1,Flow,timber,node25node18
2650,flow_timber_node3node5_t1,5.0,1,Flow,timber,node3node5
2720,flow_timber_node5node4_t1,5.0,1,Flow,timber,node5node4
2820,flow_timber_node11node3_t1,4.0,1,Flow,timber,node11node3
2940,flow_timber_node27node3_t1,1.0,1,Flow,timber,node27node3
3250,flow_timber_node23node11_t1,1.0,1,Flow,timber,node23node11
3310,flow_timber_node18node17_t1,1.0,1,Flow,timber,node18node17
3400,flow_timber_node17node11_t1,3.0,1,Flow,timber,node17node11
3460,flow_timber_node12node17_t1,1.0,1,Flow,timber,node12node17
3600,flow_timber_node21node17_t1,1.0,1,Flow,timber,node21node17


## remap

In [176]:
solution_df[['node1', 'node2']] = solution_df['edge'].str.extract(r'(node\d+)(node\d+)')

In [182]:
solution_df['node1_full'] = solution_df['node1'].map(node_mapping)
solution_df['node2_full'] = solution_df['node2'].map(node_mapping)

### Verify

#### prep df containing all infos and the solution

In [183]:
# Merge with normal node1_ID and node2_ID
merge1 = solution_df.merge(edge_attr_df, left_on="edge", right_on="EdgeID", how='inner')
merge1

,Variable,Value,period,action,type,edge,node1,node2,node1_full,node2_full,...,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade,slope,removed_nodes,node1_ID,node2_ID,EdgeID
0,C_timber_node2node20_t0,0.0,0,Construct,timber,node2node20,node2,node20,"(-16216.49208, 150851.53241)","(-16192.43465, 150699.23742)",...,3120,1113,4677,1113,2339,False,"[(-16359.36728, 150734.05721), (-16297.45468, ...",node2,node20,node2node20
1,C_timber_node2node20_t1,0.0,1,Construct,timber,node2node20,node2,node20,"(-16216.49208, 150851.53241)","(-16192.43465, 150699.23742)",...,3120,1113,4677,1113,2339,False,"[(-16359.36728, 150734.05721), (-16297.45468, ...",node2,node20,node2node20
2,C_timber_node2node20_t2,0.0,2,Construct,timber,node2node20,node2,node20,"(-16216.49208, 150851.53241)","(-16192.43465, 150699.23742)",...,3120,1113,4677,1113,2339,False,"[(-16359.36728, 150734.05721), (-16297.45468, ...",node2,node20,node2node20
3,C_timber_node2node20_t3,0.0,3,Construct,timber,node2node20,node2,node20,"(-16216.49208, 150851.53241)","(-16192.43465, 150699.23742)",...,3120,1113,4677,1113,2339,False,"[(-16359.36728, 150734.05721), (-16297.45468, ...",node2,node20,node2node20
4,C_timber_node2node20_t4,0.0,4,Construct,timber,node2node20,node2,node20,"(-16216.49208, 150851.53241)","(-16192.43465, 150699.23742)",...,3120,1113,4677,1113,2339,False,"[(-16359.36728, 150734.05721), (-16297.45468, ...",node2,node20,node2node20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,flow_wide_node16node25_t1,0.0,1,Flow,wide,node16node25,node16,node25,"(-15230.13938, 150665.38771)","(-15057.87723, 150708.68595)",...,0,0,0,0,0,False,NaN,node16,node25,node16node25
2916,flow_wide_node16node25_t2,0.0,2,Flow,wide,node16node25,node16,node25,"(-15230.13938, 150665.38771)","(-15057.87723, 150708.68595)",...,0,0,0,0,0,False,NaN,node16,node25,node16node25
2917,flow_wide_node16node25_t3,0.0,3,Flow,wide,node16node25,node16,node25,"(-15230.13938, 150665.38771)","(-15057.87723, 150708.68595)",...,0,0,0,0,0,False,NaN,node16,node25,node16node25
2918,flow_wide_node16node25_t4,0.0,4,Flow,wide,node16node25,node16,node25,"(-15230.13938, 150665.38771)","(-15057.87723, 150708.68595)",...,0,0,0,0,0,False,NaN,node16,node25,node16node25


In [184]:
# Merge with reversed node1_ID and node2_ID
merge2 = solution_df.merge(edge_attr_df, left_on=["node1", "node2"], right_on=["node2_ID", "node1_ID"], how='inner')
merge2

,Variable,Value,period,action,type,edge,node1,node2,node1_full,node2_full,...,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade,slope,removed_nodes,node1_ID,node2_ID,EdgeID
0,flow_timber_node7node1_t1,0.0,1,Flow,timber,node7node1,node7,node1,"(-15443.44238, 150603.44861)","(-15106.71098, 150581.22331)",...,1847,719,2772,719,1386,False,"[(-15167.38228, 150601.09241), (-15256.60638, ...",node1,node7,node1node7
1,flow_timber_node7node1_t2,0.0,2,Flow,timber,node7node1,node7,node1,"(-15443.44238, 150603.44861)","(-15106.71098, 150581.22331)",...,1847,719,2772,719,1386,False,"[(-15167.38228, 150601.09241), (-15256.60638, ...",node1,node7,node1node7
2,flow_timber_node7node1_t3,0.0,3,Flow,timber,node7node1,node7,node1,"(-15443.44238, 150603.44861)","(-15106.71098, 150581.22331)",...,1847,719,2772,719,1386,False,"[(-15167.38228, 150601.09241), (-15256.60638, ...",node1,node7,node1node7
3,flow_timber_node7node1_t4,0.0,4,Flow,timber,node7node1,node7,node1,"(-15443.44238, 150603.44861)","(-15106.71098, 150581.22331)",...,1847,719,2772,719,1386,False,"[(-15167.38228, 150601.09241), (-15256.60638, ...",node1,node7,node1node7
4,flow_timber_node7node1_t5,0.0,5,Flow,timber,node7node1,node7,node1,"(-15443.44238, 150603.44861)","(-15106.71098, 150581.22331)",...,1847,719,2772,719,1386,False,"[(-15167.38228, 150601.09241), (-15256.60638, ...",node1,node7,node1node7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,flow_wide_node22node16_t1,0.0,1,Flow,wide,node22node16,node22,node16,"(-15189.10959, 150864.98136)","(-15230.13938, 150665.38771)",...,0,0,0,0,0,False,NaN,node16,node22,node16node22
726,flow_wide_node22node16_t2,0.0,2,Flow,wide,node22node16,node22,node16,"(-15189.10959, 150864.98136)","(-15230.13938, 150665.38771)",...,0,0,0,0,0,False,NaN,node16,node22,node16node22
727,flow_wide_node22node16_t3,0.0,3,Flow,wide,node22node16,node22,node16,"(-15189.10959, 150864.98136)","(-15230.13938, 150665.38771)",...,0,0,0,0,0,False,NaN,node16,node22,node16node22
728,flow_wide_node22node16_t4,0.0,4,Flow,wide,node22node16,node22,node16,"(-15189.10959, 150864.98136)","(-15230.13938, 150665.38771)",...,0,0,0,0,0,False,NaN,node16,node22,node16node22


In [185]:
# Concatenate the two results and drop any duplicate rows
verify_df = pd.concat([merge1, merge2]).drop_duplicates().reset_index(drop=True)
verify_df

,Variable,Value,period,action,type,edge,node1,node2,node1_full,node2_full,...,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade,slope,removed_nodes,node1_ID,node2_ID,EdgeID
0,C_timber_node2node20_t0,0.0,0,Construct,timber,node2node20,node2,node20,"(-16216.49208, 150851.53241)","(-16192.43465, 150699.23742)",...,3120,1113,4677,1113,2339,False,"[(-16359.36728, 150734.05721), (-16297.45468, ...",node2,node20,node2node20
1,C_timber_node2node20_t1,0.0,1,Construct,timber,node2node20,node2,node20,"(-16216.49208, 150851.53241)","(-16192.43465, 150699.23742)",...,3120,1113,4677,1113,2339,False,"[(-16359.36728, 150734.05721), (-16297.45468, ...",node2,node20,node2node20
2,C_timber_node2node20_t2,0.0,2,Construct,timber,node2node20,node2,node20,"(-16216.49208, 150851.53241)","(-16192.43465, 150699.23742)",...,3120,1113,4677,1113,2339,False,"[(-16359.36728, 150734.05721), (-16297.45468, ...",node2,node20,node2node20
3,C_timber_node2node20_t3,0.0,3,Construct,timber,node2node20,node2,node20,"(-16216.49208, 150851.53241)","(-16192.43465, 150699.23742)",...,3120,1113,4677,1113,2339,False,"[(-16359.36728, 150734.05721), (-16297.45468, ...",node2,node20,node2node20
4,C_timber_node2node20_t4,0.0,4,Construct,timber,node2node20,node2,node20,"(-16216.49208, 150851.53241)","(-16192.43465, 150699.23742)",...,3120,1113,4677,1113,2339,False,"[(-16359.36728, 150734.05721), (-16297.45468, ...",node2,node20,node2node20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3645,flow_wide_node22node16_t1,0.0,1,Flow,wide,node22node16,node22,node16,"(-15189.10959, 150864.98136)","(-15230.13938, 150665.38771)",...,0,0,0,0,0,False,NaN,node16,node22,node16node22
3646,flow_wide_node22node16_t2,0.0,2,Flow,wide,node22node16,node22,node16,"(-15189.10959, 150864.98136)","(-15230.13938, 150665.38771)",...,0,0,0,0,0,False,NaN,node16,node22,node16node22
3647,flow_wide_node22node16_t3,0.0,3,Flow,wide,node22node16,node22,node16,"(-15189.10959, 150864.98136)","(-15230.13938, 150665.38771)",...,0,0,0,0,0,False,NaN,node16,node22,node16node22
3648,flow_wide_node22node16_t4,0.0,4,Flow,wide,node22node16,node22,node16,"(-15189.10959, 150864.98136)","(-15230.13938, 150665.38771)",...,0,0,0,0,0,False,NaN,node16,node22,node16node22


#### check if value in period 0 is always 0

In [186]:
verify_df[(verify_df["Value"] != 0) & (verify_df["period"] == 0)]

,Variable,Value,period,action,type,edge,node1,node2,node1_full,node2_full,...,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade,slope,removed_nodes,node1_ID,node2_ID,EdgeID


#### check if maintenance only after construct or maintain before

In [211]:
# Filter to only timber roads
timber_df = verify_df[verify_df["type"] == "timber"].copy()
timber_df["period"] = timber_df["period"].astype(int)

# Filter to only timber roads
wide_df = verify_df[verify_df["type"] == "wide"].copy()
wide_df["period"] = wide_df["period"].astype(int)

##### timber

In [209]:
# Filter for Maintain or Upgrade with Value 1 and period >= 1
check_df = timber_df[
    (timber_df["action"].isin(["Maintain", "Upgrade"])) &
    (timber_df["Value"] == 1) &
    (timber_df["period"] >= 1)
]

# Create a helper DataFrame to match against
reference_df = timber_df[
    (timber_df["action"].isin(["Construct", "Maintain"])) &
    (timber_df["Value"] == 1)
][["edge", "period"]]

# Merge with shifted period to look for t-1 match
check_df["check_period"] = check_df["period"] - 1

# Perform a left merge to see if valid previous action exists
validated_df = check_df.merge(reference_df, left_on=["edge", "check_period"], right_on=["edge", "period"], how="left", indicator=True)

# Report results
validated_df["status"] = validated_df["_merge"].map({"both": 0, "left_only": 1})

# Show rows with status
print(f'There are {validated_df[["edge", "period_x", "action", "status"]].status.sum()} rows with missing construction or maintenance in previous period')
print(validated_df[["edge", "period_x", "action", "status"]][validated_df.status==1])

There are 0.0 rows with missing construction or maintenance in previous period
Empty DataFrame
Columns: [edge, period_x, action, status]
Index: []


C:\Users\johan\AppData\Local\Temp\ipykernel_16708\2858091495.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_df["check_period"] = check_df["period"] - 1


##### wide

In [214]:
# Filter for Maintain or Upgrade with Value 1 and period >= 1
check_df = wide_df[
    (wide_df["action"].isin(["Maintain"])) &
    (wide_df["Value"] == 1) &
    (wide_df["period"] >= 1)
]

# Create a helper DataFrame to match against
reference_df = wide_df[
    (wide_df["action"].isin(["Construct", "Maintain", "Upgrade"])) &
    (wide_df["Value"] == 1)
][["edge", "period"]]

# Merge with shifted period to look for t-1 match
check_df["check_period"] = check_df["period"] - 1

# Perform left merge to validate
validated_df = check_df.merge(
    reference_df,
    left_on=["edge", "check_period"],
    right_on=["edge", "period"],
    how="left",
    indicator=True
)

# Add status column: 0 = valid, 1 = error
validated_df["status"] = validated_df["_merge"].map({"both": 0, "left_only": 1})

# Report results
print(f"There are {validated_df.status.sum()} rows with missing construction or maintenance or upgrade in previous period")
print(validated_df[["edge", "period_x", "action", "status"]][validated_df.status == 1])


There are 0.0 rows with missing construction or maintenance or upgrade in previous period
Empty DataFrame
Columns: [edge, period_x, action, status]
Index: []


## Visualize the result

### plot - helper function

In [113]:
def plot_and_save(outpath, G, name, prefix):
    fig = plt.figure(figsize=(10, 10))
    filename = os.path.join(outpath, f'{prefix}_{name}.png')

    pos = {node: node for node in G.nodes()}  # Nodes are coordinates

    # Draw graph with simple styling
    nx.draw(
        G,
        pos=pos,
        node_size=30,
        node_color="red",
        edge_color="gray",
        with_labels=False
    )

    plt.title(f"{name} | Period {prefix}")
    plt.axis("equal")
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close(fig)

In [100]:
os.makedirs("1_Preprocessed_Data/4_Solution", exist_ok=True)
outpath = '1_Preprocessed_Data/4_Solution'

In [115]:
# Filter non-zero value rows for Flow action
flow_df = solution_df[(solution_df["Value"] != 0)]

# Loop through each combination of period and type
for (period, roadtype, action), df_group in flow_df.groupby(["period", "type", "action"]):
    G = nx.Graph()
    G.add_edges_from(zip(df_group.node1_full, df_group.node2_full))

    # Include both period and type in the prefix or name
    plot_and_save(outpath, G, name=f'solution_{action}_{roadtype}', prefix=str(period))